In [11]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from lpn import utility_class

BASE_URL = "https://philipp.openai.azure.com/"
API_KEY = ""
DEPLOYMENT_NAME = "gpt-4"

llm = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
)



def extract_code(algorithm_str: str) -> str:
    """Extract code from algorithm string."""
    code = algorithm_str.split("```")[1][6:]  # 6 is the length of "python"
    return code

In [10]:
algorithm_str = """def algorithm(train_samples, train_parity, test_samples):
    return 100"""


def meta_utility(improve_str: str):
    """
    Evaluates the algorithm in improve_str to improve the algorithm in algorithm_str, 
    according to some downstream utility function. This meta-utility function can only be called n times.
    """
    n_tests = 2
    expected_utility = 0
    for _ in range(n_tests):
        try: 
            exec(improve_str, globals())
        except:
            continue
        improved_algorithm_str = improve_algorithm(algorithm_str, utility_class, llm)
        expected_utility += utility_class.func(improved_algorithm_str) / n_tests
    return expected_utility

improve_str = """def improve_algorithm(initial_solution, utility, language_model, n_calls=1):

    \"\"\"Improves a solution according to a utility function.\"\"\"
    
    expertise = "You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms."

    message = f\"\"\"Improve the following solution:

```python
{initial_solution}
```

You will be evaluated based on this score function:

```python
{utility.str}
```

You must return an improved solution. Be as creative as you can under the constraints.
Your primary improvement must be novel and non-trivial. First, propose an idea, then implement it. The solution algorithm function must have the name algorithm\"\"\"
    
    system_prompt = SystemMessagePromptTemplate.from_template(expertise)
    human_prompt = HumanMessagePromptTemplate.from_template(message)
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
    chain = LLMChain(llm=language_model, prompt=chat_prompt)
    new_solutions = []
    for _ in range(n_calls):
        new_solution = chain.run(stop=["System:"])  
        new_solution = extract_code(new_solution) 
        new_solutions.append(new_solution)
    best_solution = max(new_solutions, key=lambda x: utility.func(x))
    return best_solution"""

expected_utility = meta_utility(improve_str)
expected_utility

predictions [1 0 0 0 1 1 1 1 0 0 1 1 0 1 1 1 1 0 1 0]
predictions [1 1 1 0 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 1]
predictions [1 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 0]
predictions [0 1 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 1 1 1]


0.575

In [12]:
meta_str = """def meta_utility(improve_str: str):

    \"\"\"
    Evaluates the algorithm in improve_str to improve the algorithm in algorithm_str, 
    according to some downstream utility function. This meta-utility function can only be called n times.
    \"\"\"
    n_tests = 1
    expected_utility = 0
    for _ in range(n_tests):
        try: 
            exec(improve_str, globals())
        except:
            continue
        improved_algorithm_str = improve_algorithm(algorithm_str, utility_class, llm)
        expected_utility += utility_class.func(improved_algorithm_str) / n_tests
    return expected_utility"""


class MetaUtility():

    def __init__(self, ustr, ufunc):
        self.str = ustr
        self.func = ufunc

meta_class = MetaUtility(ustr=meta_str, ufunc=meta_utility)

In [13]:
improve_str = """def improve_algorithm(initial_solution, utility, language_model, n_calls=1):

    \"\"\"Improves a solution according to a utility function.\"\"\"
    
    expertise = "You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms."

    message = f\"\"\"Improve the following solution:

```python
{{initial_solution}}
```

You will be evaluated based on this score function:

```python
{{utility.str}}
```

You must return an improved solution. Be as creative as you can under the constraints.
Your primary improvement must be novel and non-trivial. First, propose an idea, then implement it. The solution algorithm function must have the name algorithm\"\"\"
    
    system_prompt = SystemMessagePromptTemplate.from_template(expertise)
    human_prompt = HumanMessagePromptTemplate.from_template(message)
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
    chain = LLMChain(llm=language_model, prompt=chat_prompt)
    new_solutions = []
    for _ in range(n_calls):
        new_solution = chain.run()  
        new_solution = extract_code(new_solution) 
        new_solutions.append(new_solution)
    best_solution = max(new_solutions, key=lambda x: utility.func(x))
    return best_solution"""

def improve_algorithm(initial_solution, utility, language_model, n_calls=1):

    """Improves a solution according to a utility function."""
    
    expertise = "You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms. Do not user more than 5000 characters."

    message = f"""Improve the following solution:

```python
{initial_solution}
```

You will be evaluated based on this score function:

```python
{utility.str}
```

You must return an improved solution. Be as creative as you can under the constraints.
Your primary improvement must be novel and non-trivial. First, propose an idea, then implement it."""
    system_prompt = SystemMessagePromptTemplate.from_template(expertise)
    human_prompt = HumanMessagePromptTemplate.from_template(message)
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
    chain = LLMChain(llm=language_model, prompt=chat_prompt)
    new_solutions = []
    for _ in range(n_calls):
        print('running')
        new_solution = chain.run(stop=['System:'])  
        print(new_solution)

        # new_solution = extract_code(new_solution) 
        # new_solutions.append(new_solution)
    # best_solution = max(new_solutions, key=lambda x: utility.func(x))
    # return best_solution

In [14]:
improved_improver = improve_algorithm(improve_str, meta_class, llm)

running
The current code uses a language model to generate new solutions and then selects the best one according to some utility function. However, there are a couple of areas that can be improved:

1. The same language model is used repeatedly. Since the language model is stochastic, it may not always yield the best solution on every run. It would be better to use multiple models and select the best solution across all of them.

2. The current implementation does not take into account the performance of the generated solutions. If a solution performs poorly, it is still considered for the final selection. A better approach would be to discard solutions that fall below a certain threshold.

Now, let's implement these improvements:

```python
def improve_algorithm(initial_solution, utility, language_models, n_calls=1, threshold=0.5):

    """Improves a solution according to a utility function."""
    
    expertise = "You are an expert computer science researcher and programmer, especia

In [15]:
import random

def random_walk_solver(formula, max_iter, p):
    n = max(abs(lit) for clause in formula for lit in clause)
    assignments = [False] * (n + 1)
    for _ in range(max_iter):
        unsatisfied_clauses = [clause for clause in formula if not any(assignments[abs(lit)] == (lit > 0) for lit in clause)]
        if not unsatisfied_clauses:
            return assignments
        clause_to_flip = random.choice(unsatisfied_clauses)
        if random.random() < p:
            lit_to_flip = random.choice(clause_to_flip)
        else:
            lit_to_flip = min(clause_to_flip, key=lambda lit: sum(assignments[abs(lit)] == (lit > 0) for clause in formula if lit in clause))
        assignments[abs(lit_to_flip)] = not assignments[abs(lit_to_flip)]
    return None

def algorithm(formula):
    return random_walk_solver(formula, max_iter=1000, p=0.4)
